# Цель: сгенерировать текст стилем автора, используя предобученную LLM

In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re

from google.colab import drive
from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

from warnings import filterwarnings
filterwarnings('ignore')

sns.set_style('darkgrid')

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Используется GPU: {torch.cuda.get_device_name(0)}")

Используется GPU: Tesla T4


## Доп. библиотеки 

In [2]:
!pip install Spire.Doc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 39.7 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install transformers==4.21.0 tokenizers==0.12.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 49.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 92.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.11 requires transformers>=4.33.1, but you have transformers 4.21.0 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.21.0 which 

In [16]:
!pip install -U sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.2 MB/s eta 0:00:00 0:00:01


### Очищаем .p65

In [8]:
!find . -name "*.p65" -delete

### Считываем все файлы

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


# Чтение текстов с файлов

In [4]:
import pathlib
books = pathlib.Path("/kaggle/input/bolshakov-data/new_data")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

In [5]:
files_to_process[0]

'/kaggle/input/bolshakov-data/new_data/Pril.rtf'

# Просмотр и обработка текстов

In [123]:
from spire.doc import *
from spire.doc.common import *
tokenizer_t5 = T5Tokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")

In [124]:
def split_into_chunks(text, max_tokens=512):
    """
    Разбивает текст на чанки по max_tokens токенов, сохраняя границы предложений.
    """
    sentences = re.split(r'(?<=[.!?]) +', text)  
    
    chunks = []
    current_chunk = []
    current_length = 0

    for sent in sentences:
        sent_tokens = tokenizer_t5.tokenize(sent)
        sent_token_count = len(sent_tokens)

        if sent_token_count > max_tokens:
            for i in range(0, sent_token_count, max_tokens):
                chunk = tokenizer_t5.convert_tokens_to_string(sent_tokens[i:i+max_tokens])
                chunks.append(chunk)
            continue

        if current_length + sent_token_count > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0

        current_chunk.append(sent)
        current_length += sent_token_count

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [125]:
doc = Document()
doc.LoadFromFile(files_to_process[0])
text = doc.GetText()
text = text.replace('Evaluation Warning: The document was created with Spire.Doc for Python.', '')
text = text.replace('\t', '')
clean_text = re.sub(
    r'\d+\r\n|\r\n|\x0c|УДК.*|ББК.*|Издание.*|– М.*|ISBN.*|©.*',
    '\n', 
    text
)

In [126]:
clean_text[:100]

'\nПРИЛОЖЕНИЕ\n\n1. Обратная связь как основной принцип саморегуляции систем\nи устойчивости управления\n\n'

In [127]:
final_chunks = []
paragraphs = [p.strip() for p in clean_text.split('\n') if p.strip()]
for paragraph in paragraphs:
    if len(paragraph) < 100:
        continue
    chunks = split_into_chunks(paragraph)
    final_chunks.extend(chunks)

In [128]:
for i, chunk in enumerate(final_chunks):
    if i <= 10:
        print(f"Часть {i+1}, токенов: {len(tokenizer.tokenize(chunk))}")
        print(chunk[:200] + "...\n")

Часть 1, токенов: 375
В функционировании социально-политических систем первостепенную роль играют обратные связи. Более того, любые системы с самоорганизацией способны успешно функционировать лишь при наличии соответствующ...

Часть 2, токенов: 183
Кратко напомним, что такое управление с обратными связями и какие обратные связи существуют. Под управлением с обратной связью понимают такое управление, когда результаты процесса воздействуют на его ...

Часть 3, токенов: 230
Стимулирующие обратные связи делят на положительные и отрицательные в зависимости от того, усиливают или ослабляют они управляемый  процесс. Положительная обратная связь приводит ко все большему откло...

Часть 4, токенов: 127
Общий для всех систем способ их организации прост: на выходе какого-либо процесса (иногда в специальном элементе сравнения) сравниваются заранее запланированный и реально достигнутый результаты. Сигна...

Часть 5, токенов: 188
Обратные связи могут осуществляться как с помощью специализированных э

# Генерирование тестовой выборки с помощью перефразеров-переводчиков



## Функция переплексии - метрика, которая вычисляет схожесть текстов по смыслу, применяя модель Сбера. И остальные метрики

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

In [12]:
mname = 'ai-forever/rugpt3small_based_on_gpt2'
gpt_tokenizer = AutoTokenizer.from_pretrained(mname)
gpt_model = AutoModelForCausalLM.from_pretrained(mname)
gpt_model.cuda();

In [13]:
def get_gpt2_ppl_corpus(test_textes, aggregate=True, sep='\n', max_length=1024):
    lls, weights = [], []
    for text in tqdm(test_textes):
        encodings = gpt_tokenizer(f'{sep}{text}{sep}', return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        input_ids = encodings.input_ids.to(gpt_model.device)
        target_ids = input_ids.clone()

        w = max(0, len(input_ids[0]) - 1)
        if w > 0:
            with torch.no_grad():
                output = gpt_model(input_ids, labels=target_ids)
                likelihood = output[0]
                ll = likelihood.item()
        else:
            ll = 0
        lls.append(ll)
        weights.append(w)
    likelihoods, weights = np.array(lls), np.array(weights)
    if aggregate:
        return sum(likelihoods * weights) / sum(weights)
    return likelihoods, weights

------

In [64]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
sentences = ["Hello World", "Привет Мир"]
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
print(embeddings)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-0.0714, -0.0772, -0.0305,  ...,  0.0136, -0.0402, -0.0245],
        [-0.0644, -0.0687, -0.0521,  ...,  0.0139, -0.0362, -0.0213]])


In [ ]:
def similarity_with_model()

## Валидация работы предобученных переводчиков на одном файле 

In [103]:
from transformers import FSMTModel, FSMTTokenizer, FSMTForConditionalGeneration

In [104]:
# ПРИМЕР С БИБЛИОТЕКИ

from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-ru-en"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

input = "Макс занимается языком программирования Джава"
input_ids = tokenizer.encode(input, return_tensors="pt")
outputs = model.generate(input_ids)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded) 

Max is engaged in Java programming language


In [105]:
tokenizer_tr = FSMTTokenizer.from_pretrained("facebook/wmt19-en-ru")
model_tr = FSMTForConditionalGeneration.from_pretrained("facebook/wmt19-en-ru")
inverse_tokenizer_tr = FSMTTokenizer.from_pretrained("facebook/wmt19-ru-en")
inverse_model_tr = FSMTForConditionalGeneration.from_pretrained("facebook/wmt19-ru-en")
model.cuda();
inverse_model.cuda();

In [208]:
def paraphrase(text, gram=4, num_beams=5, max_length=1024, **kwargs):
    input_ids = inverse_tokenizer.encode(text, return_tensors='pt', padding=True, truncations=True, max_length=max_length)
    print(input_ids)
    
    with torch.no_grad():
        output = inverse_model.generate(input_ids.to(inverse_model.device), num_beams=num_beams, **kwargs)
        
    other_lang = inverse_tokenizer.decode(output[0], skip_special_tokens=True)
    input_ids = input_ids[0, :-1].tolist()
    bad_words_ids = [input_ids[i:(i+gram)] for i in range(len(input_ids)-gram)]
    
    input_ids = tokenizer.encode(other_lang, return_tensors='pt', padding=True, truncations=True, max_length=max_length)
    
    with torch.no_grad():
        output = model.generate(input_ids.to(model.device), num_beams=num_beams, bad_words_ids=bad_words_ids, **kwargs)
    
    decode = tokenizer.decode(output[0], skip_special_tokens=True)
    return decode

In [209]:
%%time
text = final_chunks[0]
print(paraphrase(text, gram=3, do_sample=False))
new_text = paraphrase(text, gram=3, do_sample=False)

Keyword arguments {'truncations': True} not recognized.


tensor([[   22,  3141,  2804,  1816,    14,  1057,   902,  6118,  6151,   151,
           527,  6086,  2308,   650,    69,     5,  2979,    49,     4,  2658,
           130,    12,  1559,  2873,  9617,  4765,  3141,   565,   676,    53,
          3528,  2907,  7603,  2308,   568,  2800,     5,  2661,     4, 14642,
            31,   940,  8178,    31,   263,   902,    25,  3939,     4,  4146,
          1002,     4,  9911,    25,  2881,  1963,  1447,     8,   158,  7107,
          2946,     4,    33,     6, 12408, 15021,  1646,  4486,  4064,     5,
          3947,    61,    14,  3268,   130,     4, 11156,     4,   156,   103,
           325,  6115,   834,    21,  1559,  5238,  2514,     4,   211,    33,
          2658,  6948,     7,   343,  1339,   318,   357,   676,  1287,    31,
          2738,   167,   902,     6,     7,  5176,  1934, 16336,  7504,  9112,
           249,   130,    16,   774,  1394,  1026,    16,     5, 13300,  1279,
             4,  4600,   374,  2946,  5159,   902,  

Keyword arguments {'truncations': True} not recognized.
Keyword arguments {'truncations': True} not recognized.


Например, некоторые из самых сложных систем - социальных, биологических,
tensor([[   22,  3141,  2804,  1816,    14,  1057,   902,  6118,  6151,   151,
           527,  6086,  2308,   650,    69,     5,  2979,    49,     4,  2658,
           130,    12,  1559,  2873,  9617,  4765,  3141,   565,   676,    53,
          3528,  2907,  7603,  2308,   568,  2800,     5,  2661,     4, 14642,
            31,   940,  8178,    31,   263,   902,    25,  3939,     4,  4146,
          1002,     4,  9911,    25,  2881,  1963,  1447,     8,   158,  7107,
          2946,     4,    33,     6, 12408, 15021,  1646,  4486,  4064,     5,
          3947,    61,    14,  3268,   130,     4, 11156,     4,   156,   103,
           325,  6115,   834,    21,  1559,  5238,  2514,     4,   211,    33,
          2658,  6948,     7,   343,  1339,   318,   357,   676,  1287,    31,
          2738,   167,   902,     6,     7,  5176,  1934, 16336,  7504,  9112,
           249,   130,    16,   774,  1394,  1026,    16, 

Keyword arguments {'truncations': True} not recognized.


CPU times: user 2.71 s, sys: 4.5 ms, total: 2.71 s
Wall time: 2.71 s


In [210]:
text

'В функционировании социально-политических систем первостепенную роль играют обратные связи. Более того, любые системы с самоорганизацией способны успешно функционировать лишь при наличии соответствующего набора обратных связей. Например, одни из наиболее сложных из таких систем — социальные, биологические, экологические — основаны именно на этом принципе функционирования, как и современные автоматизированные технические устройства. Социально-политические системы, безусловно, должны быть отнесены к самоорганизующимся, так как любые имеющиеся в них органы управления являются лишь одним из элементов этих систем и в оптимальном варианте тоже подлежат организации системы “изнутри”. Следовательно, успешность функционирования общественных систем должна существенно зависеть от того, как организована в них сеть обратных связей. Можно ли говорить об оптимальной организации этих связей, существуют ли здесь какие-либо закономерности? Актуальность этого вопроса многократно возрастает в связи с про

In [109]:
print(f'Значение переплекcии для абзаца 1: {get_gpt2_ppl_corpus(new_text)}')

  0%|          | 0/443 [00:00<?, ?it/s]

Значение переплекcии для абзаца 1: 6.350892123121482


-------

In [110]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [140]:
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-paraphraser").to(device)

In [251]:
len(tokenizer_t5.tokenize(final_chunks[6]))

83

In [252]:
text = final_chunks[6]
input_ids = tokenizer.encode("paraphrase: " + text, return_tensors="pt").to(device)
outputs = model.generate(
    input_ids,
    do_sample=True,           # ← Включить сэмплирование
    temperature=1,          # Случайность (0.5-1.2)
    top_k=50,                 # Ограничить выбор топ-50 токенами
    top_p=0.95,               # Nucleus sampling
    max_length=83+50,
    min_length=83,
    no_repeat_ngram_size=3
)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [253]:
decoded

'Практика ежедневного наблюдения напоминает, что люди, не только живущие, понимают, как и как следует сделать, чтобы улучшить ситуацию в стране, обладают всей необходимой информации, но в каком-то случае не смогут реализовать своих доброго намерения. (Вопросы, вопросы, опросы, консультации)'

In [254]:
final_chunks[6]

'Практика повседневной жизни свидетельствует, что сплошь и рядом люди знают реальное положение дел, во многом понимают, что и как надо делать для улучшения ситуации в стране, обладают всей необходимой для этого информацией, но почему-то не могут реализовать свои добрые намерения.'

## Полная генерация всей тестовой выборки 